In [1]:
import os

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
import pandas as pd
from functools import partial
from sklearn.model_selection import StratifiedKFold
import numpy as np

In [4]:
tf.__version__

'2.4.1'

In [5]:
gpus = tf.config.list_physical_devices('GPU')

In [6]:
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
if gpus:
    
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs, ", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        print(e)

1 Physical GPUs,  1 Logical GPU


In [8]:
strategy = tf.distribute.get_strategy()

# DATA LOADING

In [9]:
train_df = pd.read_csv('data/train_concat.csv')
train_df = train_df.iloc[:len(train_df)//2, :]

In [10]:
def parse_image(data, train):
    image_name = tf.strings.join(['data/train/train/', data['image_name'], '.jpg'])
    label = data['target']
    
    if train:
        image = tf.io.read_file(image_name)
    image = tf.image.decode_jpeg(image)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image, label = image_augmentations(image, label)
#    image = tf.image.resize(image, [224, 224])
    return image, label

In [11]:
def get_tfrecord_size(tfrecord):
    return sum(1 for _ in tfrecord)

In [12]:
def image_augmentations(image,label):
    img_size = (224, 224)
    
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    
    if tf.random.uniform([], 0, 1.0, dtype = tf.float32) > 0.75:
        image = tf.image.transpose(image)
    
    probablity_rotation = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
    if probablity_rotation > 0.75:
        image = tf.image.rot90(image, k = 3)
    elif probablity_rotation > 0.5:
        image = tf.image.rot90(image, k = 2)
    elif probablity_rotation > 0.25:
        image = tf.image.rot90(image, k = 1)
        
    if tf.random.uniform([], 0, 1.0, dtype = tf.float32) >= 0.4:
        image = tf.image.random_saturation(image, lower = 0.8, upper = 1.2)
    if tf.random.uniform([], 0, 1.0, dtype = tf.float32) >= 0.4:
        image = tf.image.random_contrast(image, lower = 0.8, upper = 1.2)
    if tf.random.uniform([], 0, 1.0, dtype = tf.float32) >= 0.4:
        image = tf.image.random_brightness(image, max_delta = 0.1)
    
    probability_cropping = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
    if probability_cropping > 0.7:
        if probability_cropping > 0.9:
            image = tf.image.central_crop(image, central_fraction = 0.7)
        elif probability_cropping > 0.8:
            image = tf.image.central_crop(image, central_fraction = 0.8)
        else:
            image = tf.image.central_crop(image, central_fraction = 0.9)
    elif probability_cropping > 0.5:
        crop_size = tf.random.uniform([], int(img_size[0] * 0.8), img_size[0], dtype = tf.int32)
        image = tf.image.random_crop(image, size = [crop_size, crop_size, 3])

    image = tf.image.resize(image, size = img_size)
#    image = tf.reshape(image, [*img_size, 3])
    
    return image, label

In [13]:
def get_dataset(df, train, batchsize):
    tf_ds = tf.data.Dataset.from_tensor_slices(dict(df[['image_name', 'target']]))
    return tf_ds.map(partial(parse_image, train=True), num_parallel_calls=tf.data.experimental.AUTOTUNE).shuffle(1000).batch(batchsize).prefetch(tf.data.experimental.AUTOTUNE)

# EFFICIENTNET B0

In [14]:
from tensorflow.keras.applications import EfficientNetB0

In [15]:
def efficientnetb0():
    
    K.reset_uids()
    
    inputs = keras.layers.Input(shape=(224, 224, 3))
    
    effnet = EfficientNetB0(include_top=False, input_shape=(224, 224, 3))
    
    effnet = effnet(inputs)
    pooling = keras.layers.GlobalAveragePooling2D()(effnet)
    x = keras.layers.Dense(1000, activation='relu')(pooling)
    x = keras.layers.BatchNormalization()(x)
    dropout = keras.layers.Dropout(0.4)(x)
    outputs = keras.layers.Dense(1, activation='sigmoid')(dropout)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    
    return model

# LOSS

In [16]:
def focal_loss(y_true, y_pred, alpha=1.0, gamma=2.0):
    
    alpha = tf.convert_to_tensor(alpha, dtype=K.floatx())
    gamma = tf.convert_to_tensor(gamma, dtype=K.floatx())
    
#     y_true = tf.convert_to_tensor(y_true)
#     y_pred = tf.convert_to_tensor(y_pred)
    
    loss = keras.losses.binary_crossentropy(y_true, y_pred)
    
    pt = K.exp(-loss)
    
    loss = alpha*(1-pt)**gamma*loss
    return K.mean(loss)
    
    
    

# SCHEDULER

In [21]:
def lrfn(epoch, bs=24, epochs=10):
    
    LR_START = 1e-6
    LR_MAX = 2e-4
    LR_FINAL = 1e-6
    LR_RAMPUP_EPOCHS = 4
    LR_SUSTAIN_EPOCHS = 0
    DECAY_EPOCHS = epochs  - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS - 1
    LR_EXP_DECAY = (LR_FINAL / LR_MAX) ** (1 / (epochs - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS - 1))

    if epoch < LR_RAMPUP_EPOCHS:
        lr = LR_START + (LR_MAX + LR_START) * (epoch / LR_RAMPUP_EPOCHS) ** 2.5
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        epoch_diff = epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS
        decay_factor = (epoch_diff / DECAY_EPOCHS) * math.pi
        decay_factor= (tf.math.cos(decay_factor).numpy() + 1) / 2        
        lr = LR_FINAL + (LR_MAX - LR_FINAL) * decay_factor

    return lr

# TRAIN

In [22]:
skf = StratifiedKFold(n_splits=5, random_state=999, shuffle=True)

In [23]:
folds = skf.split(X=np.zeros(len(train_df)), y=train_df.loc[: ,['target']])

In [ ]:
for fold, (train_idx, val_idx) in enumerate(folds,1):
    
    train_data = get_dataset(train_df.iloc[train_idx].reset_index(drop=True), True, 24)
    validation_data = get_dataset(train_df.iloc[val_idx].reset_index(drop=True), True, 24)
    train_size = get_tfrecord_size(train_data)
    
    with strategy.scope():
        
        model = efficientnetb0()
        
        opt = keras.optimizers.Adam(learning_rate=0.001)
#        loss = keras.losses.binary_crossentropy

        model.compile(loss=focal_loss, optimizer=opt, metrics=['acc', tf.keras.metrics.AUC()])
    
    model.summary()
    
    cb_checkpoint = tf.keras.callbacks.ModelCheckpoint("backup.h5",monitor="val_loss", verbose=1, save_best_only=True)
    cb_earlystop = keras.callbacks.EarlyStopping(monitor='val_auc', mode='max', 
                                                 patience=4, restore_best_weights=True, verbose=1)

#    cb_lr = tf.keras.callbacks.ReduceLROnPlateau(patience=3)
    cb_lr = tf.keras.callbacks.LearningRateScheduler(lambda epoch: lrfn(epoch), verbose=1)
    
    params = {"epochs":10,
              "validation_data": validation_data,
              "callbacks": [cb_earlystop, cb_checkpoint, cb_lr]} 
    
    
    history = model.fit(train_data, **params)
        